Jupyter notebook based on kernel GTendolkar to improve parameters and try to get better results.

In [24]:
import numpy as np
import pandas as pd
import scipy as scp
import csv

train_master = pd.read_csv('./input/train.csv')
test_master = pd.read_csv('./input/test.csv')
train_master.describe()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,5.952120e+05,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,...,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,7.438036e+05,0.036448,1.900378,1.358943,4.423318,0.416794,0.405188,0.393742,0.257033,0.163921,...,5.441382,1.441918,2.872288,7.539026,0.122427,0.627840,0.554182,0.287182,0.349024,0.153318
std,4.293678e+05,0.187401,1.983789,0.664594,2.699902,0.493311,1.350642,0.488579,0.436998,0.370205,...,2.332871,1.202963,1.694887,2.746652,0.327779,0.483381,0.497056,0.452447,0.476662,0.360295
min,7.000000e+00,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.719915e+05,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.435475e+05,0.000000,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,1.115549e+06,0.000000,3.000000,2.000000,6.000000,1.000000,0.000000,1.000000,1.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.488027e+06,1.000000,7.000000,4.000000,11.000000,1.000000,6.000000,1.000000,1.000000,1.000000,...,19.000000,10.000000,13.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
def gini(y, pred):
    fpr, tpr, thr = metrics.roc_curve(y, pred, pos_label=1)
    g = 2 * metrics.auc(fpr, tpr) -1
    return g

In [26]:
from sklearn.model_selection import StratifiedKFold

n_splits = 10
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=7)

In [27]:
import numpy as np
from sklearn import metrics

def encode_cat_features(train_df, test_df, cat_cols, target_col_name, smoothing=1):
    prior = train_df[target_col_name].mean()
    probs_dict = {}
    for c in cat_cols:
        probs = train_df.groupby(c, as_index=False)[target_col_name].mean()
        probs['counts'] = train_df.groupby(c, as_index=False)[target_col_name].count()[[target_col_name]]
        probs['smoothing'] = 1 / (1 + np.exp(-(probs['counts'] - 1) / smoothing))
        probs['enc'] = prior * (1 - probs['smoothing']) + probs['target'] * probs['smoothing']
        probs_dict[c] = probs[[c,'enc']]
    return probs_dict

In [28]:
import lightgbm as lgb
import numpy as np
import pandas as pd

np.random.seed(3)
model_scores = {}

# Drop binary columns with almost all zeros. 
# Why now? Just follow along for now. We have a lot of experimentation to be done
train = train_master.drop(['ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_13_bin'],axis=1)
test = test_master.drop(['ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_13_bin'],axis=1)

# Drop calculated features
# But WHY??? 
# Because we are assuming that tree can generate any complicated function 
# of base features and calculated features add no more information
# Is this assumption valid? Results will tell
calc_columns = [s for s in list(train_master.columns.values) if '_calc' in s]
train = train.drop(calc_columns, axis=1)  
test = test.drop(calc_columns, axis=1)

# Get categorical columns for encoding later
categorical_columns = [s for s in list(train_master.columns.values) if '_cat' in s]
target_column = 'target'

# Replace missing values with NaN
train = train.replace(-1, np.nan)
test = test.replace(-1, np.nan)

# Initialize DS to store validation fold predictions
y_val_fold = np.empty(len(train))

# Initialize DS to store test predictions with aggregate model and individual models
y_test = np.zeros(len(test))
y_test_model_1 = np.zeros(len(test))
y_test_model_2 = np.zeros(len(test))
y_test_model_3 = np.zeros(len(test))

for fold_number, (train_ids, val_ids) in enumerate(
    folds.split(train.drop(['id',target_column], axis=1), 
                train[target_column])):
    
    X = train.iloc[train_ids]
    X_val = train.iloc[val_ids]
    X_test = test
    
    # Encode categorical variables using training fold
    encoding_dict = encode_cat_features(X, X_val, categorical_columns, target_column)
    
    for c, encoding in encoding_dict.items():
        X = pd.merge(X, encoding[[c,'enc']], how='left', on=c, sort=False,suffixes=('', '_'+c))
        X = X.drop(c, axis = 1)
        X = X.rename(columns = {'enc':'enc_'+c})
        
        X_test = pd.merge(X_test, encoding[[c,'enc']], how='left', on=c, sort=False,suffixes=('', '_'+c))
        X_test = X_test.drop(c, axis = 1)
        X_test = X_test.rename(columns = {'enc':'enc_'+c})
        
        X_val = pd.merge(X_val, encoding[[c,'enc']], how='left', on=c, sort=False,suffixes=('', '_'+c))
        X_val = X_val.drop(c, axis = 1)
        X_val = X_val.rename(columns = {'enc':'enc_'+c})
        
    # Seperate target column and remove id column from all
    y = X[target_column]
    X = X.drop(['id',target_column], axis=1)
    X_test = X_test.drop('id', axis=1)
    y_val = X_val[target_column]
    X_val = X_val.drop(['id',target_column], axis=1)
    
    # Upsample data in training folds
    ids_to_duplicate = pd.Series(y == 1)
    X = pd.concat([X, X.loc[ids_to_duplicate]], axis=0)
    y = pd.concat([y, y.loc[ids_to_duplicate]], axis=0)
    # Again Upsample (total increase becomes 4 times)
    X = pd.concat([X, X.loc[ids_to_duplicate]], axis=0)
    y = pd.concat([y, y.loc[ids_to_duplicate]], axis=0)
    
    # Shuffle after concatenating duplicate rows
    # We cannot use inbuilt shuffles since both dataframes have to be shuffled in sync
    shuffled_ids = np.arange(len(X))
    np.random.shuffle(shuffled_ids)
    X = X.iloc[shuffled_ids]
    y = y.iloc[shuffled_ids]
    
    # Feature Selection goes here
    # TODO
    
    # Define parameters of GBM as explained before for 3 trees
    params_1 = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'max_depth': 3,
        'learning_rate': 0.05,
        'feature_fraction': 1,
        'bagging_fraction': 1,
        'bagging_freq': 10,
        'verbose': 0
    }
    params_2 = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'max_depth': 4,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.9,
        'bagging_freq': 2,
        'verbose': 0
    }
    params_3 = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'max_depth': 5,
        'learning_rate': 0.05,
        'feature_fraction': 0.3,
        'bagging_fraction': 0.7,
        'bagging_freq': 10,
        'verbose': 0
    }
    
    # Create appropriate format for training and evaluation data
    lgb_train = lgb.Dataset(X, y)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
    
    # Create the 3 classifiers with 1000 rounds and a window of 100 for early stopping
    clf_1 = lgb.train(params_1,lgb_train, num_boost_round=1000,
                      valid_sets=lgb_eval, early_stopping_rounds=100, verbose_eval=50)
    clf_2 = lgb.train(params_2,lgb_train, num_boost_round=1000,
                      valid_sets=lgb_eval, early_stopping_rounds=100, verbose_eval=50)
    clf_3 = lgb.train(params_3,lgb_train, num_boost_round=1000,
                      valid_sets=lgb_eval, early_stopping_rounds=100, verbose_eval=50)
    
    # Predict raw scores for validation ids
    # At each fold, 1/10th of the training data get scores
    y_val_fold[val_ids] = (clf_1.predict(X_val, raw_score=True)+
                           clf_2.predict(X_val, raw_score=True)+
                           clf_3.predict(X_val, raw_score=True)) / 3

    # Predict and average over folds, raw scores for test data
    y_test += (clf_1.predict(X_test, raw_score=True)+
               clf_2.predict(X_test, raw_score=True)+
               clf_3.predict(X_test, raw_score=True)) / (3*n_splits)
    y_test_model_1 += clf_1.predict(X_test, raw_score=True) / n_splits
    y_test_model_2 += clf_2.predict(X_test, raw_score=True) / n_splits
    y_test_model_3 += clf_3.predict(X_test, raw_score=True) / n_splits
    
    # Display fold predictions
    # Gini requires only order and therefore raw scores need not be scaled
    print("Fold %2d : %.9f" % (fold_number + 1, gini(y_val, y_val_fold[val_ids])))
    
# Display aggregate predictions
# Gini requires only order and therefore raw scores need not be scaled
print("Average score over all folds: %.9f" % gini(train_master[target_column], y_val_fold))

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.6295
[100]	valid_0's auc: 0.634919
[150]	valid_0's auc: 0.636065
[200]	valid_0's auc: 0.63728
[250]	valid_0's auc: 0.637661
[300]	valid_0's auc: 0.637865
[350]	valid_0's auc: 0.637623
[400]	valid_0's auc: 0.637589
Early stopping, best iteration is:
[303]	valid_0's auc: 0.637922
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.632789
[100]	valid_0's auc: 0.636587
[150]	valid_0's auc: 0.638363
[200]	valid_0's auc: 0.638781
[250]	valid_0's auc: 0.638731
[300]	valid_0's auc: 0.638656
Early stopping, best iteration is:
[218]	valid_0's auc: 0.639014
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.63314
[100]	valid_0's auc: 0.636723
[150]	valid_0's auc: 0.638186
[200]	valid_0's auc: 0.638789
[250]	valid_0's auc: 0.638213
Early stopping, best iteration is:
[198]	valid_0's auc: 0.638813
Fold  1 : 0.278339921
Training until validation score

[150]	valid_0's auc: 0.632496
[200]	valid_0's auc: 0.634265
[250]	valid_0's auc: 0.634668
[300]	valid_0's auc: 0.634684
[350]	valid_0's auc: 0.635409
[400]	valid_0's auc: 0.635042
[450]	valid_0's auc: 0.635445
Early stopping, best iteration is:
[368]	valid_0's auc: 0.635612
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.625632
[100]	valid_0's auc: 0.630207
[150]	valid_0's auc: 0.632658
[200]	valid_0's auc: 0.633717
[250]	valid_0's auc: 0.634727
[300]	valid_0's auc: 0.635679
[350]	valid_0's auc: 0.635942
[400]	valid_0's auc: 0.635978
[450]	valid_0's auc: 0.63565
Early stopping, best iteration is:
[355]	valid_0's auc: 0.636024
Fold  7 : 0.272943927
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.6221
[100]	valid_0's auc: 0.633107
[150]	valid_0's auc: 0.637235
[200]	valid_0's auc: 0.639263
[250]	valid_0's auc: 0.640392
[300]	valid_0's auc: 0.640832
[350]	valid_0's auc: 0.641362
[400]	valid_0's auc: 0.641584
[450]	v

MemoryError: 

In [ ]:
temp = y_test
# Scale the raw scores to range [0.0, 1.0]
temp = np.add(temp,abs(min(temp)))/max(np.add(temp,abs(min(temp))))

df = pd.DataFrame(columns=['id', 'target'])
df['id']=test_master['id']
df['target']=temp
df.to_csv('lgbtestwithprobs.csv', index=False, float_format="%.9f")
df.shape
